# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


#### What test is appropriate for this problem? Does CLT apply?
The most appropriate test for this problem would likely be the two-sample test, since we can use it to compare "white-sounding" names and "black-sounding" names and their callback rates. CLT may apply here, but it really depends on if the callback rates are unbiased or not (which is basically what we are exploring). If we were to add more and more points, we may in fact discover that the distribution does not approach a normal curve and that it is skewed because of racial discrimination. But in theory, CLT should apply.
#### What are the null and alternate hypotheses?
The null hypothesis is that there is NO significant difference between callback rates for different names. The alternate hypothesis is that names do somehow affect the callback rates. If our p-value is small (< 0.05) then we reject the null hypothesis and we might consider that there is indeed a difference.

In [7]:
w = data[data.race=='w']
b = data[data.race=='b']

In [9]:
w_callback = w['call']
b_callback = b['call']

In [12]:
# frequentist approach
w_std = w_callback.std()
b_std = b_callback.std()

mean_diff = np.abs(np.mean(w_callback) - np.mean(b_callback))
print('white std:', w_std, '\nblack std:', b_std, '\nmean-diff:', mean_diff)

std_err = np.sqrt(w_std**2/len(w_callback) + b_std**2/len(b_callback))
print('error:', std_err)

white std: 0.2953455150127411 
black std: 0.24564945697784424 
mean-diff: 0.03203285485506058
error: 0.007784906919813793


In [17]:
# confidence interval
conf_int = [mean_diff - 1.96*std_err, mean_diff + 1.96*std_err]
print('95% confidence:', conf_int)

95% confidence: [0.016774437292225546, 0.04729127241789561]


In [13]:
# p-value
t, p = stats.ttest_ind(w_callback, b_callback)
print('stats:', t, '\np val:', p)

stats: 4.114705290861751 
p val: 3.940802103128886e-05


In [14]:
# boostrapping
replicates = np.empty(10000)

for i in range(10000):
    combined = np.random.permutation(np.concatenate((w_callback, b_callback)))
    w_permutation = combined[:len(w_callback)]
    b_permutation = combined[len(w_callback):]
    replicates[i] = np.abs(np.mean(w_permutation) - np.mean(b_permutation))

p_val = np.sum(replicates > mean_diff)/10000
print('p val:', p_val)

p val: 0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

For both tests, the p-value is very low, meaning we would reject the null hypothesis. This would suggest that race is at least somewhat related to callback rates, at least from our comparison of the two variables only. However, we cannot simply claim this will full confidence without further testing of other variables.

Without testi